In [1]:
using PyPlot
using PyCall
using CSV,DataFrames
pushfirst!(PyVector(pyimport("sys")["path"]), "")
@pyimport product_quad as pq
include("sweep2d_2.jl")

(array([0.08151814, 0.08151814, 0.08151814, 0.08151814, 0.08151814,
       0.08151814, 0.08151814, 0.08151814, 0.04348186, 0.04348186,
       0.04348186, 0.04348186, 0.04348186, 0.04348186, 0.04348186,
       0.04348186]), array([ 0.86884614,  0.86884614,  0.35988786,  0.35988786, -0.35988786,
       -0.35988786, -0.86884614, -0.86884614,  0.46967645,  0.46967645,
        0.19454636,  0.19454636, -0.19454636, -0.19454636, -0.46967645,
       -0.46967645]), array([ 0.35988786, -0.35988786,  0.86884614, -0.86884614,  0.86884614,
       -0.86884614,  0.35988786, -0.35988786,  0.19454636, -0.19454636,
        0.46967645, -0.46967645,  0.46967645, -0.46967645,  0.19454636,
       -0.19454636]))
0.9999999999999999


RT_Solve_DMD (generic function with 1 method)

In [2]:
Nx = 50
Ny = 5
Lx = 0.00023 
# Lx = 0.4
# Lx = 0.035
Ly = 10
hx = Lx/Nx
hy = Ly/Ny

dx = hx
#make x
x = range(-Lx/2+dx/2,stop=Lx/2-dx/2,length=Nx)#linspace(-Lx/2+dx/2,(Lx/2-dx/2),Nx)

Nord = 2
w,etax,etay = pq.prod_quad(Nord)
angles = length(w)
T0 = 80e-3
# T0=1

boundaryx = zeros(angles) .+  a * c * T0^4  #80e-3#+ 1/np.sum(w)
boundaryy = zeros(angles) .+ 0#+ 1/np.sum(w)

sigma = zeros((Nx,Ny,4)) .+ 1
Q = zeros((Nx,Ny,4,angles))
source_spots = findall(x-> abs.(x)<= 0.5,x)
Q[source_spots,:,:,:] .= 1.0/sum(w)*(a*c) 


psi_init = Q*0
Ti = 1.0e-10
T_init = zeros((Nx,Ny,4)) .+ Ti
psi_init = Q*0 .+ a*c*Ti^4
v = c
Tfinal = 5e-13 * 10^8 
# Tfinal = 1
# Tfinal = 0.000005 / 10.
dt = Tfinal/10000 #hx/c
cv_val =  8310 * 1e-16 * (11604) * 1e3
# cv_val = 0.3
println("cv = ", cv_val)
Cv(T,density) = cv_val  .+ 0.0*4*a*T.^3
eos(T,density) = cv_val .*T
inveos(e,density) = e/cv_val #(e/a).^.25
L2tol = 1e-8
Linftol = 1e-5
phi_sol = nothing
Q_func(t) = ifelse(t <= (10/c),Q.*0,0 .*Q)
n = 3
sigma0 = 107991.
# sigma0 = 300
sigma_func(T,density) = sigma0 * ( T ./ T0 ) .^ (-n) .* sigma     #107991
density = 0.01
# density = 0.3
K = 3
Rits = 3
psi,phi_sol, T, times, iteration = RT_Solve(Nx,Ny,hx,hy,psi_init,T_init,Nord,density,sigma_func,Q_func,Cv,
                        inveos,eos,v,dt,Tfinal,
             boundaryx, boundaryy,LOUD = 1);


cv = 9.642924e-6


LoadError: MethodError: no method matching +(::Int64, ::Array{Float64, 3})
For element-wise addition, use broadcasting with dot syntax: scalar .+ array
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  +(::T, [91m::T[39m) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:87
[0m  +(::Union{Int16, Int32, Int64, Int8}, [91m::BigInt[39m) at gmp.jl:537
[0m  ...

In [3]:
x,y,phi_flat = flatten_phi((T[:,:,:,end-1]),Nx,Ny,hx,hy)
plot(x .- Lx/2,phi_flat[:,Int64(round(Ny/2))])
CSV.write("Marshak.csv", (data=phi_flat[:,Int64(round(Ny/2))],))

LoadError: UndefVarError: T not defined

In [4]:
function xi(n_)
    if n_ == 0
        return 1.232
    elseif n_ == 3
        return 1.119935
    end
end

wave_estimate = xi(n) * sqrt(Tfinal) / sqrt(3*(n+4)*sigma0*cv_val / (8*c*a*T0^(3)))

0.0002198736565392481

In [5]:
println(size(phi_sol))
x,y,phi_flat = flatten_phi(phi_sol[:,:,:,end-1],Nx,Ny,hx,hy)


plot(-(x .- Lx),((phi_flat[:,Int64(round(Ny/2))])/a/c).^.25, lw = 3)


# plot(-(x .- Lx),(phi_flat[:,Int64(round(Ny/2))]), lw=2)


CSV.write("phi_cv1.csv", (data=phi_flat[:,Int64(round(Ny/2))],))
CSV.write("x_cv1.csv", (data= x[:,1] .- Lx/2 ,))

x,y,phi_flat = flatten_phi((T[:,:,:,end-1]),Nx,Ny,hx,hy)

plot(-(x .- Lx),phi_flat[:,Int64(round(Ny/2))])


scatter(x= (wave_estimate), y=0.0)
# scatter(x= (0.0002), y=0.0)



LoadError: MethodError: no method matching size(::Nothing)
[0mClosest candidates are:
[0m  size([91m::Union{QR, LinearAlgebra.QRCompactWY, QRPivoted}[39m) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/qr.jl:581
[0m  size([91m::Union{QR, LinearAlgebra.QRCompactWY, QRPivoted}[39m, [91m::Integer[39m) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/qr.jl:580
[0m  size([91m::Union{Cholesky, CholeskyPivoted}[39m) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/cholesky.jl:514
[0m  ...

In [6]:
a*c

4.1157256

In [7]:
c

299.98

In [8]:
sum(w)

1.0

In [9]:
T0

0.08